# Homework 9
12110418 
庄子鲲
## Problem 1
Using Python and Numpy, write a class named SVMClassifier, which implements the SVM algorithm having slack variables and kernels such as Polynomial,Gaussian, and Sigmoid (using cvxopt package to solve the quadratic programing problem for Lagrange multipliers).

For a svm with slack variables and kernels:
Optimize function:
$$
\max_{a_i\ge 0}\{\sum_{i=1}^{N}\alpha _i-\frac{1}{2}\sum_{i,j=1}^{N}t^{(i)}t^{(j)}\alpha^i \alpha ^j K(\mathbf x^{(i)},\mathbf x^{(j)}) \}\\
$$
where
$$
0\le \alpha^{(i)}\le -\xi^{(i)}\\
\xi^{(i)}\ge0
$$
Linear kernel:
$$
K(\mathbf x^{(i)},\mathbf x^{(j)})=\mathbf x^{(i)^{T}} \mathbf x^{(j)}
$$
$d^{\text {th}}$ degree of Polynomial kernel:
$$
K(\mathbf x^{(i)},\mathbf x^{(j)})=(\mathbf x^{(i)}\mathbf x^{(j)}+1)^d
$$
Gaussian kernel:
$$
K(\mathbf x^{(i)},\mathbf x^{(j)})=\exp(-\frac{||\mathbf x^{(i)}-\mathbf x^{(j)}||^2}{2\sigma ^2})
$$
Sigmoid kernel:
$$
K(\mathbf x^{(i)},\mathbf x^{(j)})=\tanh (\beta (\mathbf x^{(i)^{T}}\mathbf x^{(j)}))
$$
Output:
$$
y=\text {sign}[b+\mathbf x\cdot (\sum_{i=1}^{(N)}\alpha_it^{(i)}\mathbf x^{(i)})]
$$


In [53]:
import numpy as np
from cvxopt import matrix, solvers

class SVMClassifier:
    def __init__(self, kernel='linear', degree=3, alpha_threshold=1e-5,gamma=None, coef0=0.0, C=1.0):
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.C = C
        self.alpha_threshold = alpha_threshold
        self.alpha = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.bias = None

    def linear_kernel(self, x1, x2):
        return np.dot(x1, x2.T)
    
    def poly_kernel(self, x1, x2):
        return (np.dot(x1, x2.T) + self.coef0)**self.degree
    
    def rbf_kernel(self, x1, x2):
        return np.exp(-((x1[:, np.newaxis] - x2)**2).sum(axis=2)/(2 * self.gamma**2))
    
    def sigmoid_kernel(self, x1, x2):
        return np.tanh(self.gamma * np.dot(x1, x2.T) + self.coef0)
    
    def fit(self, X, y):
        if self.kernel == 'linear':
            self.K = self.linear_kernel(X, X)
        elif self.kernel == 'poly':
            self.K = self.poly_kernel(X, X)
        elif self.kernel == 'rbf':
            if self.gamma is None:
                self.gamma = 1.0 / X.shape[1]  # Default gamma
            self.K = self.rbf_kernel(X, X)
        elif self.kernel == 'sigmoid':
            self.K = self.sigmoid_kernel(X, X)

        n_samples, n_features = X.shape
        P = matrix(np.outer(y, y) * self.K, tc='d')
        q = matrix(-np.ones(n_samples), tc='d')
        G = matrix(np.vstack((np.eye(n_samples), -np.eye(n_samples))), tc='d')
        h = matrix(np.hstack((self.C * np.ones(n_samples), np.zeros(n_samples))), tc='d')
        A = matrix(y.reshape(1, -1), tc='d')
        b = matrix(0.0, tc='d')

        # Solve the quadratic programming problem
        solution = solvers.qp(P, q, G, h, A, b)

        # Extract Lagrange multipliers from the solution
        self.alpha = np.array(solution['x']).flatten()
        # Support vectors have non-zero Lagrange multipliers
        sv_indices = np.where(self.alpha > self.alpha_threshold)[0]
        self.support_vectors = X[sv_indices]
        self.support_vector_labels = y[sv_indices]
        print('number of support vector: ',self.support_vectors.shape[0])
        self.alpha = self.alpha.reshape(-1,1)
        self.alpha=self.alpha[sv_indices]
        # Compute the bias term
        if self.kernel == 'linear':
            self.bias = np.mean(self.support_vector_labels-np.dot(self.linear_kernel(self.support_vectors,self.support_vectors), (self.alpha * self.support_vector_labels)))     
        elif self.kernel == 'poly':
            self.bias = np.mean(self.support_vector_labels-np.dot(self.poly_kernel(self.support_vectors,self.support_vectors), (self.alpha * self.support_vector_labels)))
        elif self.kernel=='rbf':
            self.bias = np.mean(self.support_vector_labels-np.dot(self.rbf_kernel(self.support_vectors,self.support_vectors),(self.alpha * self.support_vector_labels)))
        elif self.kernel=='sigmoid':
            self.bias = np.mean(self.support_vector_labels-np.dot(self.sigmoid_kernel(self.support_vectors,self.support_vectors),(self.alpha * self.support_vector_labels)))
        
        
    def predict(self, X):
        if self.kernel == 'linear':
            decision_function = np.dot(self.linear_kernel(X,self.support_vectors), (self.alpha * self.support_vector_labels)) + self.bias
        elif self.kernel == 'poly':
            decision_function = np.dot(self.poly_kernel(X,self.support_vectors), (self.alpha * self.support_vector_labels)) + self.bias
        elif self.kernel == 'rbf':
            decision_function = np.dot(self.rbf_kernel(X,self.support_vectors),(self.alpha * self.support_vector_labels)) + self.bias
        elif self.kernel == 'sigmoid':
            decision_function = np.dot(self.sigmoid_kernel(X,self.support_vectors),(self.alpha * self.support_vector_labels)) + self.bias

        # Predict using the sign of the decision function
        return np.sign(decision_function)


## Problem 2
Consider the dataset of letter recognition (named letter-recognition.data).The dataset has 20,000 samples, for which the first column indicates theclass(A~Z,totally 26 classes), and the rest columns indicate 16 features as described in the following table. For this dataset, use SVM to do a binary classification for letter‘C’or non-‘C’class.

1. Linear Kernel:

In [54]:
svm_linear = SVMClassifier(kernel='linear', gamma=0.1, C=1.0)
# Load data from the file
data = np.loadtxt("letter-recognition.data", dtype=str, delimiter=',')
# Extract labels and features
y = np.array([1 if label == "C" else -1 for label in data[:3000, 0]])
y=y.reshape(-1,1)
X = data[:3000, 1:].astype(int)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape[0],'training data is used to train')
svm_linear.fit(X_train, y_train)
predictions = svm_linear.predict(X_test)
print(X_test.shape[0],'testing data to be test')
print('accuracy:', accuracy_score(y_test, predictions))

2400 training data is used to train
     pcost       dcost       gap    pres   dres
 0: -2.7929e+02 -5.0035e+03  3e+04  3e+00  1e-11
 1: -1.8392e+02 -2.7396e+03  5e+03  3e-01  8e-12
 2: -1.6411e+02 -1.2470e+03  2e+03  1e-01  6e-12
 3: -1.4639e+02 -1.0070e+03  1e+03  8e-02  5e-12
 4: -1.3218e+02 -6.3803e+02  8e+02  4e-02  5e-12
 5: -1.2359e+02 -4.6493e+02  5e+02  2e-02  4e-12
 6: -1.2221e+02 -2.8184e+02  2e+02  8e-03  4e-12
 7: -1.2275e+02 -2.3608e+02  1e+02  5e-03  4e-12
 8: -1.2552e+02 -1.8358e+02  7e+01  2e-03  4e-12
 9: -1.2631e+02 -1.8003e+02  6e+01  1e-03  4e-12
10: -1.2643e+02 -1.7016e+02  5e+01  6e-04  4e-12
11: -1.2787e+02 -1.6118e+02  4e+01  3e-04  4e-12
12: -1.2855e+02 -1.6002e+02  3e+01  3e-04  4e-12
13: -1.3116e+02 -1.5410e+02  2e+01  2e-04  4e-12
14: -1.3132e+02 -1.5282e+02  2e+01  1e-04  4e-12
15: -1.3368e+02 -1.4786e+02  1e+01  7e-05  4e-12
16: -1.3419e+02 -1.4700e+02  1e+01  5e-05  4e-12
17: -1.3519e+02 -1.4465e+02  1e+01  2e-05  4e-12
18: -1.3601e+02 -1.4316e+02  7e+00

2. Polynomial Kernel:

In [55]:
svm_poly = SVMClassifier(kernel='poly', alpha_threshold=1e-10,degree=3, gamma=0.1, C=1.0)
svm_poly.fit(X_train, y_train)
predictions = svm_poly.predict(X_test)
print('accuracy:', accuracy_score(y_test, predictions))

     pcost       dcost       gap    pres   dres
 0: -5.7607e+01 -3.9301e+03  2e+04  2e+00  2e-06
 1: -3.4795e+01 -1.8461e+03  3e+03  3e-01  2e-06
 2: -1.7125e+01 -8.9159e+02  2e+03  1e-01  1e-06
 3: -1.0456e+01 -5.6253e+02  1e+03  7e-02  1e-06
 4: -5.4415e-01 -2.6114e+02  4e+02  2e-02  4e-07
 5: -7.7377e-03 -1.2862e+01  2e+01  9e-04  4e-08
 6: -3.0346e-03 -1.5850e+00  2e+00  1e-04  5e-09
 7: -1.9960e-03 -2.9432e-01  4e-01  2e-05  1e-09
 8: -1.0008e-03 -7.8224e-02  1e-01  4e-06  3e-10
 9: -4.8212e-04 -3.4432e-02  5e-02  1e-06  2e-10
10: -2.5371e-04 -1.1747e-02  2e-02  4e-07  7e-11
11: -1.4527e-04 -4.7776e-03  6e-03  1e-07  4e-11
12: -7.8857e-05 -2.2889e-03  3e-03  6e-08  2e-11
13: -4.8948e-05 -9.7840e-04  1e-03  2e-08  1e-11
14: -5.3992e-05 -4.1926e-04  5e-04  7e-09  9e-12
15: -5.8278e-05 -1.7974e-04  1e-04  6e-10  1e-11
16: -7.2909e-05 -1.3397e-04  6e-05  2e-10  9e-12
17: -8.1590e-05 -1.0639e-04  2e-05  2e-16  1e-11
18: -8.9078e-05 -9.4524e-05  5e-06  2e-16  1e-11
19: -9.0815e-05 -9.21

3. Guassian Kernel:

In [60]:
svm_rbf = SVMClassifier(kernel='rbf',gamma=0.1, C=1.0)
svm_rbf.fit(X_train, y_train)
predictions = svm_rbf.predict(X_test)
print('accuracy:', accuracy_score(y_test, predictions))

     pcost       dcost       gap    pres   dres
 0: -1.6213e+02 -3.3969e+03  1e+04  2e+00  1e-16
 1: -1.4415e+02 -1.4385e+03  2e+03  7e-02  7e-16
 2: -1.4029e+02 -2.1626e+02  8e+01  5e-04  6e-16
 3: -1.4046e+02 -1.4139e+02  9e-01  6e-06  1e-16
 4: -1.4052e+02 -1.4053e+02  2e-02  8e-08  1e-16
 5: -1.4052e+02 -1.4052e+02  3e-04  8e-10  8e-17
 6: -1.4052e+02 -1.4052e+02  3e-06  8e-12  5e-17
Optimal solution found.
number of support vector:  2400
accuracy: 0.9566666666666667


4. Sigmoid Kernel:

In [61]:
svm_sigmoid = SVMClassifier(kernel='sigmoid',gamma=0.1, C=1.0)
svm_sigmoid.fit(X_train, y_train)
predictions = svm_sigmoid.predict(X_test)
print('accuracy:', accuracy_score(y_test, predictions))

     pcost       dcost       gap    pres   dres
 0: -3.6496e+02 -4.9750e+03  3e+04  3e+00  2e-13
 1: -2.0534e+02 -2.5896e+03  3e+03  1e-01  2e-13
 2: -1.8972e+02 -3.3922e+02  2e+02  3e-04  2e-14
 3: -1.8995e+02 -1.9150e+02  2e+00  3e-06  4e-14
 4: -1.9000e+02 -1.9002e+02  2e-02  3e-08  2e-14
 5: -1.9000e+02 -1.9000e+02  2e-04  3e-10  2e-14
Optimal solution found.
number of support vector:  2400
accuracy: 0.9566666666666667
